<a href="https://colab.research.google.com/github/LeonardoRoig/TECH_5/blob/main/CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast, json, re
import warnings
warnings.filterwarnings('ignore')

# TRATAMENTO PROSPECTS

In [185]:
# Load the JSON data
with open("/content/prospects.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

In [186]:


# Process the dictionary to flatten the nested structure
records = []
for prospects_id, prospects_data in data.items():
    # Ensure nested data is in dictionary format
    titulo = prospects_data.get("titulo", {}) if isinstance(prospects_data.get("titulo"), dict) else {}
    modalidade = prospects_data.get("modalidade", {}) if isinstance(prospects_data.get("modalidade"), dict) else {}
    prospects_list = prospects_data.get("prospects", []) # 'prospects' is a list of dicts

    # Create a base dictionary for the current prospect
    merged_data = {"prospects_id": prospects_id}

    # Add items from 'titulo' and 'modalidade' with prefixes
    merged_data.update({f"titulo_{k}": v for k, v in titulo.items()})
    merged_data.update({f"modalidade_{k}": v for k, v in modalidade.items()})

    # Handle the list of prospects within 'prospects'
    # If there are multiple prospects in the list, this approach will only take the first one
    # If you need to handle multiple prospects per prospects_id, the structure of the output DataFrame would need to change (e.g., one row per prospect)
    if prospects_list:
        # Assuming we only take the first prospect in the list for flattening into a single row
        first_prospect = prospects_list[0] if isinstance(prospects_list[0], dict) else {}
        merged_data.update({f"prospects_{k}": v for k, v in first_prospect.items()})
    else:
         # Add placeholder for expected prospect columns if the list is empty
         # Based on the applicants data, common keys might be 'nome', 'codigo', 'situacao'
         merged_data.update({f"prospects_nome": None, f"prospects_codigo": None, f"prospects_situacao": None})


    records.append(merged_data)

# Create a DataFrame from the flattened records
df_flat = pd.DataFrame(records)

# Display the head of the flattened DataFrame to check the structure
print("Head of the flattened DataFrame:")
display(df_flat.head())

Head of the flattened DataFrame:


,prospects_id,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador,prospects_situacao
0,4530,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,NaN
1,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,NaN
2,4532,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,4533,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,NaN
4,4534,Ana Luiza Vieira,26361,Documentação PJ,28-04-2021,11-05-2021,Aguardando confirmação de inicio _,Manuella Carvalho,NaN


In [187]:
df_flat.head()

,prospects_id,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador,prospects_situacao
0,4530,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,NaN
1,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,NaN
2,4532,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,4533,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,NaN
4,4534,Ana Luiza Vieira,26361,Documentação PJ,28-04-2021,11-05-2021,Aguardando confirmação de inicio _,Manuella Carvalho,NaN


In [188]:
df_flat.columns = (
    pd.Series(df_flat.columns)
    .astype(str)
    .str.strip()
    .str.lower()
    .str.normalize('NFKD')  # remove acentos
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace(r'\s+', '_', regex=True)
)

In [189]:
df_flat.head()

,prospects_id,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador,prospects_situacao
0,4530,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,NaN
1,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,NaN
2,4532,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,4533,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira,NaN
4,4534,Ana Luiza Vieira,26361,Documentação PJ,28-04-2021,11-05-2021,Aguardando confirmação de inicio _,Manuella Carvalho,NaN


In [196]:
# 5) Tipagem: datas (dia primeiro), valores monetários, inteiros
# Datas: qualquer coluna que contenha 'data'
for c in [c for c in df_flat.columns if 'data' in c]:
    df_flat[c] = pd.to_datetime(df_flat[c], dayfirst=True, errors='coerce')

# Função para converter dinheiro/bruto em float
money_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['prospects_comentario', 'valor', 'sal', 'remun', 'preco', 'custo'])]
def money_to_float(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip()
    if s in {'-', ''}:
        return np.nan
    # remove R$, espaços e separadores de milhar; troca vírgula por ponto
    s = s.replace('R$', '').replace('r$', '').replace(' ', '').replace('.', '').replace(',', '.')
    # pega o primeiro número válido
    m = re.search(r'-?\d+(\.\d+)?', s)
    return float(m.group(0)) if m else np.nan

for c in money_like_cols:
    df_flat[c] = df_flat[c].map(money_to_float)

In [197]:
# Inteiros evidentes (ex: limites, cargas, etc.)
int_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['limite', 'carga', 'qtd', 'quant', 'horas'])]
for c in int_like_cols:
    # só converte se não for já numérico
    if not pd.api.types.is_integer_dtype(df_flat[c]):
        df_flat[c] = pd.to_numeric(df_flat[c], errors='coerce').astype('Int64')

In [198]:
# 6) String hygiene
str_cols = df_flat.select_dtypes(include=['object']).columns
df_flat[str_cols] = df_flat[str_cols].apply(lambda s: s.astype(str).str.strip().replace({'nan': np.nan}))

In [199]:
# 7) Remover colunas totalmente nulas e duplicadas
df_flat = df_flat.loc[:, df_flat.notna().any(axis=0)]
df_flat = df_flat.loc[:, ~df_flat.columns.duplicated()]

In [200]:
# 8) Ordenar colunas (id primeiro)
cols = ['prospects_id'] + [c for c in df_flat.columns if c != 'prospects_id']
df_final = df_flat[cols]

In [201]:
df_final.head()

,prospects_id,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador
0,4530,José Vieira,25632,Encaminhado ao Requisitante,2021-03-25,25-03-2021,72.0,Ana Lívia Moreira
1,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,2021-03-17,12-04-2021,12.0,Juliana Cassiano
2,4532,None,None,NaN,NaT,NaN,NaN,NaN
3,4533,Arthur Almeida,26338,Contratado pela Decision,2021-04-29,18-05-2021,NaN,Stella Vieira
4,4534,Ana Luiza Vieira,26361,Documentação PJ,2021-04-28,11-05-2021,NaN,Manuella Carvalho


In [202]:
df_final.isnull().sum()

,0
prospects_id,0
prospects_nome,0
prospects_codigo,0
prospects_situacao_candidado,2943
prospects_data_candidatura,2943
prospects_ultima_atualizacao,2943
prospects_comentario,12330
prospects_recrutador,2943


In [203]:
pd.set_option('display.max_columns', None)
df_final.tail(5)

,prospects_id,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador
14217,14218,Dr. Vitor Hugo Silva,40384,Entrevista Técnica,2025-02-26,27-02-2025,27.0,Laura Pacheco
14218,14219,None,None,NaN,NaT,NaN,NaN,NaN
14219,14220,Ana Cardoso,16828,Desistiu,2025-02-26,28-02-2025,NaN,Elisa Nunes
14220,14221,Maria Eduarda Cassiano,49190,Prospect,2025-02-26,26-02-2025,NaN,Luna Correia
14221,14222,None,None,NaN,NaT,NaN,NaN,NaN


In [204]:
colunas = [
    "prospects_id", "prospects_nome", "prospects_codigo", "prospects_situacao_candidado",
    "prospects_data_candidatura", "prospects_ultima_atualizacao", "prospects_comentario",
    "prospects_recrutador"
]

prospects = df_final[colunas]


In [205]:
for col in [
    "prospects_id", "prospects_nome", "prospects_codigo", "prospects_situacao_candidado",
    "prospects_data_candidatura", "prospects_ultima_atualizacao", "prospects_comentario",
    "prospects_recrutador"
]:
    prospects[col] = prospects[col].fillna("N/A").replace({"": "N/A", "nan": "N/A"})


In [206]:
prospects.head()

,prospects_id,prospects_nome,prospects_codigo,prospects_situacao_candidado,prospects_data_candidatura,prospects_ultima_atualizacao,prospects_comentario,prospects_recrutador
0,4530,José Vieira,25632,Encaminhado ao Requisitante,2021-03-25 00:00:00,25-03-2021,72.0,Ana Lívia Moreira
1,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,2021-03-17 00:00:00,12-04-2021,12.0,Juliana Cassiano
2,4532,None,None,N/A,N/A,N/A,N/A,N/A
3,4533,Arthur Almeida,26338,Contratado pela Decision,2021-04-29 00:00:00,18-05-2021,N/A,Stella Vieira
4,4534,Ana Luiza Vieira,26361,Documentação PJ,2021-04-28 00:00:00,11-05-2021,N/A,Manuella Carvalho


In [207]:
# Salvar o DataFrame vagas em um arquivo CSV
prospects.to_csv("prospects_processadas.csv", index=False)
print("DataFrame 'prospects' salvo como 'prospects_processadas.csv'")

DataFrame 'prospects' salvo como 'prospects_processadas.csv'


# TRATAMENTO APPLICANTS

In [58]:
df_raw = pd.read_json("/content/applicants.json")

In [59]:
# 1) Garantir que cada célula seja um dict (pode vir como string)
def to_dict(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    s = str(x)
    # tenta Python-literal
    try:
        return ast.literal_eval(s)
    except Exception:
        pass
    # tenta JSON
    try:
        return json.loads(s)
    except Exception:
        return {}

df_raw = df_raw.applymap(to_dict)

In [60]:
# 2) Transpor: cada linha vira um job_id
# Transpose df_raw so job_id is in the index and the nested sections are columns
df_t = df_raw.T
df_t.index.name = "applicants_id"

In [70]:
df_t.head()

,infos_basicas,informacoes_pessoais,informacoes_profissionais,formacao_e_idiomas,cargo_atual,cv_pt,cv_en
applicants_id,,,,,,,
31000,"{'telefone_recado': '', 'telefone': '(11) 9704...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},{},{}
31001,"{'telefone_recado': '', 'telefone': '(11) 9372...",{'data_aceite': 'Cadastro anterior ao registro...,{'titulo_profissional': 'Analista Administrati...,{'nivel_academico': 'Ensino Superior Incomplet...,{},{},{}
31002,"{'telefone_recado': '', 'telefone': '(11) 9239...",{'data_aceite': 'Cadastro anterior ao registro...,{'titulo_profissional': 'Administrativo | Fina...,{'nivel_academico': 'Ensino Superior Completo'...,{},{},{}
31003,"{'telefone_recado': '', 'telefone': '(11) 9810...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': 'Área administrativa',...",{'nivel_academico': 'Ensino Superior Incomplet...,{},{},{}
31004,"{'telefone_recado': '', 'telefone': '(11) 9251...",{'data_aceite': 'Cadastro anterior ao registro...,"{'titulo_profissional': '', 'area_atuacao': ''...","{'nivel_academico': '', 'nivel_ingles': '', 'n...",{},{},{}


In [77]:
def merge_sections(row):
    # Extrai os campos, usando {} como padrão
    infos_basicas = row.get("infos_basicas", {})
    informacoes_pessoais = row.get("informacoes_pessoais", {})
    informacoes_profissionais = row.get("informacoes_profissionais", {})
    formacao_e_idiomas = row.get("formacao_e_idiomas", {})
    cargo_atual = row.get("cargo_atual", {})
    cv_pt = row.get("cv_pt", {})
    cv_en = row.get("cv_en", {})

    # Verifica se cada campo é um dicionário antes de acessar .items()
    info = {f"inf_{k}": v for k, v in infos_basicas.items()} if isinstance(infos_basicas, dict) else {}
    pessoal = {f"inf_{k}": v for k, v in informacoes_pessoais.items()} if isinstance(informacoes_pessoais, dict) else {}
    profissional = {f"inf_{k}": v for k, v in informacoes_profissionais.items()} if isinstance(informacoes_profissionais, dict) else {}
    formacao = {f"formacao_{k}": v for k, v in formacao_e_idiomas.items()} if isinstance(formacao_e_idiomas, dict) else {}
    cargo = {f"cargo_{k}": v for k, v in cargo_atual.items()} if isinstance(cargo_atual, dict) else {}
    cv_pt_items = {f"cv_pt_{k}": v for k, v in cv_pt.items()} if isinstance(cv_pt, dict) else {}
    cv_en_items = {f"cv_en_{k}": v for k, v in cv_en.items()} if isinstance(cv_en, dict) else {}

    # Junta tudo em um único dicionário
    return {**info, **pessoal, **profissional, **formacao, **cargo, **cv_pt_items, **cv_en_items}


In [78]:
# 4) Flatten: expand nested dicts to columns
records = df_t.apply(merge_sections, axis=1)
df_flat = pd.DataFrame(list(records), index=df_t.index).reset_index()

In [79]:
df_flat.columns = (
    pd.Series(df_flat.columns)
    .astype(str)
    .str.strip()
    .str.lower()
    .str.normalize('NFKD')  # remove acentos
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace(r'\s+', '_', regex=True)
)

In [81]:
# 5) Tipagem: datas (dia primeiro), valores monetários, inteiros
# Datas: qualquer coluna que contenha 'data'
for c in [c for c in df_flat.columns if 'data' in c]:
    df_flat[c] = pd.to_datetime(df_flat[c], dayfirst=True, errors='coerce')

# Função para converter dinheiro/bruto em float
money_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['valor', 'sal', 'remun', 'preco', 'custo'])]
def money_to_float(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip()
    if s in {'-', ''}:
        return np.nan
    # remove R$, espaços e separadores de milhar; troca vírgula por ponto
    s = s.replace('R$', '').replace('r$', '').replace(' ', '').replace('.', '').replace(',', '.')
    # pega o primeiro número válido
    m = re.search(r'-?\d+(\.\d+)?', s)
    return float(m.group(0)) if m else np.nan

for c in money_like_cols:
    df_flat[c] = df_flat[c].map(money_to_float)

In [82]:
# Inteiros evidentes (ex: limites, cargas, etc.)
int_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['limite', 'carga', 'qtd', 'quant', 'horas'])]
for c in int_like_cols:
    # só converte se não for já numérico
    if not pd.api.types.is_integer_dtype(df_flat[c]):
        df_flat[c] = pd.to_numeric(df_flat[c], errors='coerce').astype('Int64')

In [83]:
# 6) String hygiene
str_cols = df_flat.select_dtypes(include=['object']).columns
df_flat[str_cols] = df_flat[str_cols].apply(lambda s: s.astype(str).str.strip().replace({'nan': np.nan}))

In [84]:
# 7) Remover colunas totalmente nulas e duplicadas
df_flat = df_flat.loc[:, df_flat.notna().any(axis=0)]
df_flat = df_flat.loc[:, ~df_flat.columns.duplicated()]

In [86]:
# 8) Ordenar colunas (id primeiro)
cols = ['applicants_id'] + [c for c in df_flat.columns if c != 'applicants_id']
df_final = df_flat[cols]

In [87]:
df_final.head()

,applicants_id,inf_telefone_recado,inf_telefone,inf_objetivo_profissional,inf_data_criacao,inf_inserido_por,inf_email,inf_local,inf_sabendo_de_nos_por,inf_data_atualizacao,inf_codigo_profissional,inf_nome,inf_data_aceite,inf_cpf,inf_fonte_indicacao,inf_email_secundario,inf_data_nascimento,inf_telefone_celular,inf_sexo,inf_estado_civil,inf_pcd,inf_endereco,inf_skype,inf_url_linkedin,inf_facebook,inf_titulo_profissional,inf_area_atuacao,inf_conhecimentos_tecnicos,inf_certificacoes,inf_outras_certificacoes,inf_remuneracao,inf_nivel_profissional,formacao_nivel_academico,formacao_nivel_ingles,formacao_nivel_espanhol,formacao_outro_idioma,formacao_instituicao_ensino_superior,formacao_cursos,formacao_ano_conclusao,inf_download_cv,inf_qualificacoes,inf_experiencias,formacao_outro_curso,cargo_id_ibrati,cargo_email_corporativo,cargo_cargo_atual,cargo_projeto_atual,cargo_cliente,cargo_unidade,cargo_data_admissao,cargo_data_ultima_promocao,cargo_nome_superior_imediato,cargo_email_superior_imediato
0,31000,,(11) 97048-2708,,2021-11-10 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,2021-11-10 07:29:49,31000,Carolina Aparecida,NaT,,:,,NaT,(11) 97048-2708,,,,,,,,,,,,,NaN,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,2021-11-10 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,2021-11-11 11:10:31,31001,Eduardo Rios,NaT,,Outros: Contato do RH,,1994-12-28,(11) 93723-4396,Feminino,Solteiro,Não,são paulo,,,,Analista Administrativo,Administrativa,,,,19000.0,,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,2021-11-10 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,2021-11-10 11:42:36,31002,Pedro Henrique Carvalho,NaT,,Anúncio:,,1988-12-12,(11) 92399-9824,Feminino,Solteiro,Não,são paulo,,,,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,25000.0,,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,,Administração de Empresas,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,2021-11-10 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,2021-11-10 16:04:51,31003,Thiago Barbosa,NaT,,Site de Empregos: Infojobs,,1992-05-08,(11) 98100-1727,Feminino,Casado,Não,são paulo,,,,Área administrativa,Administrativa,,,,11000.0,,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
4,31004,,(11) 92517-2678,,2021-11-10 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,2021-11-10 09:22:03,31004,Diogo das Neves,NaT,,:,,1969-12-31,(11) 92517-2678,,,,,,,,,,,,,NaN,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [88]:
df_final.isnull().sum()

,0
applicants_id,0
inf_telefone_recado,0
inf_telefone,0
inf_objetivo_profissional,0
inf_data_criacao,19
inf_inserido_por,0
inf_email,0
inf_local,0
inf_sabendo_de_nos_por,0
inf_data_atualizacao,18


In [89]:
pd.set_option('display.max_columns', None)
df_final.tail(5)

,applicants_id,inf_telefone_recado,inf_telefone,inf_objetivo_profissional,inf_data_criacao,inf_inserido_por,inf_email,inf_local,inf_sabendo_de_nos_por,inf_data_atualizacao,inf_codigo_profissional,inf_nome,inf_data_aceite,inf_cpf,inf_fonte_indicacao,inf_email_secundario,inf_data_nascimento,inf_telefone_celular,inf_sexo,inf_estado_civil,inf_pcd,inf_endereco,inf_skype,inf_url_linkedin,inf_facebook,inf_titulo_profissional,inf_area_atuacao,inf_conhecimentos_tecnicos,inf_certificacoes,inf_outras_certificacoes,inf_remuneracao,inf_nivel_profissional,formacao_nivel_academico,formacao_nivel_ingles,formacao_nivel_espanhol,formacao_outro_idioma,formacao_instituicao_ensino_superior,formacao_cursos,formacao_ano_conclusao,inf_download_cv,inf_qualificacoes,inf_experiencias,formacao_outro_curso,cargo_id_ibrati,cargo_email_corporativo,cargo_cargo_atual,cargo_projeto_atual,cargo_cliente,cargo_unidade,cargo_data_admissao,cargo_data_ultima_promocao,cargo_nome_superior_imediato,cargo_email_superior_imediato
42477,5995,,(11) 92622-2088,Crm / Líder de Projetos,2018-03-14 16:35:14,pelo próprio candidato,gabrielly_novaes@gmail.com,,Outros,2018-03-14 16:35:14,5995,Gabrielly Novaes,NaT,,Outros: Importação,,NaT,(11) 92622-2088,,,,,,,,Crm / Líder de Projetos,,,,,0.0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
42478,5996,,(41) 93009-5141,Crm,2018-03-14 16:38:28,pelo próprio candidato,sr._gabriel_rios@gmail.com,,Outros,2018-03-14 16:38:28,5996,Sr. Gabriel Rios,NaT,,Outros: Importação,,NaT,(41) 93009-5141,,,,,,,,Crm,,,,,0.0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
42479,5997,,(11) 97837-3941,Crm,2018-03-14 16:41:52,pelo próprio candidato,dr._mathias_da_cunha@gmail.com,,Outros,2018-03-14 16:41:52,5997,Dr. Mathias da Cunha,NaT,,Outros: Importação,,NaT,(11) 97837-3941,,,,,,,,Crm,,,,,0.0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
42480,5998,,(61) 96806-8075,Crm,2018-03-14 16:42:49,pelo próprio candidato,bruna_vieira@hotmail.com,,Outros,2018-03-14 16:42:49,5998,Bruna Vieira,NaT,,Outros: Importação,,NaT,(61) 96806-8075,,,,,,,,Crm,,,,,0.0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
42481,5999,,(15) 95637-4857,Crm,2018-03-14 16:47:27,pelo próprio candidato,heitor_mendonça@hotmail.com,,Outros,2018-03-14 16:47:27,5999,Heitor Mendonça,NaT,,Outros: Importação,,NaT,(15) 95637-4857,,,,,,,,Crm,,,,,0.0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [92]:
colunas = [
    "applicants_id", "inf_telefone_recado", "inf_telefone", "inf_objetivo_profissional",
    "inf_data_criacao", "inf_inserido_por", "inf_email", "inf_local", "inf_sabendo_de_nos_por",
    "inf_data_atualizacao", "inf_codigo_profissional", "inf_nome", "inf_data_aceite", "inf_cpf",
    "inf_fonte_indicacao", "inf_email_secundario", "inf_data_nascimento", "inf_telefone_celular",
    "inf_sexo", "inf_estado_civil", "inf_pcd", "inf_endereco", "inf_skype", "inf_url_linkedin",
    "inf_facebook", "inf_titulo_profissional", "inf_area_atuacao", "inf_conhecimentos_tecnicos",
    "inf_certificacoes", "inf_outras_certificacoes", "inf_remuneracao", "inf_nivel_profissional",
    "formacao_nivel_academico", "formacao_nivel_ingles", "formacao_nivel_espanhol",
    "formacao_outro_idioma", "formacao_instituicao_ensino_superior", "formacao_cursos",
    "formacao_ano_conclusao", "inf_download_cv", "inf_qualificacoes", "inf_experiencias",
    "formacao_outro_curso", "cargo_id_ibrati", "cargo_email_corporativo", "cargo_cargo_atual",
    "cargo_projeto_atual", "cargo_cliente", "cargo_unidade", "cargo_data_admissao",
    "cargo_data_ultima_promocao", "cargo_nome_superior_imediato", "cargo_email_superior_imediato"
]

applicants = df_final[colunas]


In [99]:
for col in [
    "applicants_id", "inf_telefone_recado", "inf_telefone", "inf_objetivo_profissional",
    "inf_data_criacao", "inf_inserido_por", "inf_email", "inf_local", "inf_sabendo_de_nos_por",
    "inf_data_atualizacao", "inf_codigo_profissional", "inf_nome", "inf_data_aceite", "inf_cpf",
    "inf_fonte_indicacao", "inf_email_secundario", "inf_data_nascimento", "inf_telefone_celular",
    "inf_sexo", "inf_estado_civil", "inf_pcd", "inf_endereco", "inf_skype", "inf_url_linkedin",
    "inf_facebook", "inf_titulo_profissional", "inf_area_atuacao", "inf_conhecimentos_tecnicos",
    "inf_certificacoes", "inf_outras_certificacoes", "inf_remuneracao", "inf_nivel_profissional",
    "formacao_nivel_academico", "formacao_nivel_ingles", "formacao_nivel_espanhol",
    "formacao_outro_idioma", "formacao_instituicao_ensino_superior", "formacao_cursos",
    "formacao_ano_conclusao", "inf_download_cv", "inf_qualificacoes", "inf_experiencias",
    "formacao_outro_curso", "cargo_id_ibrati", "cargo_email_corporativo", "cargo_cargo_atual",
    "cargo_projeto_atual", "cargo_cliente", "cargo_unidade", "cargo_data_admissao",
    "cargo_data_ultima_promocao", "cargo_nome_superior_imediato", "cargo_email_superior_imediato"
]:
    applicants[col] = applicants[col].fillna("N/A").replace({"": "N/A", "nan": "N/A"})


In [100]:
applicants.head()

,applicants_id,inf_telefone_recado,inf_telefone,inf_objetivo_profissional,inf_data_criacao,inf_inserido_por,inf_email,inf_local,inf_sabendo_de_nos_por,inf_data_atualizacao,inf_codigo_profissional,inf_nome,inf_data_aceite,inf_cpf,inf_fonte_indicacao,inf_email_secundario,inf_data_nascimento,inf_telefone_celular,inf_sexo,inf_estado_civil,inf_pcd,inf_endereco,inf_skype,inf_url_linkedin,inf_facebook,inf_titulo_profissional,inf_area_atuacao,inf_conhecimentos_tecnicos,inf_certificacoes,inf_outras_certificacoes,inf_remuneracao,inf_nivel_profissional,formacao_nivel_academico,formacao_nivel_ingles,formacao_nivel_espanhol,formacao_outro_idioma,formacao_instituicao_ensino_superior,formacao_cursos,formacao_ano_conclusao,inf_download_cv,inf_qualificacoes,inf_experiencias,formacao_outro_curso,cargo_id_ibrati,cargo_email_corporativo,cargo_cargo_atual,cargo_projeto_atual,cargo_cliente,cargo_unidade,cargo_data_admissao,cargo_data_ultima_promocao,cargo_nome_superior_imediato,cargo_email_superior_imediato
0,31000,N/A,(11) 97048-2708,N/A,2021-11-10 07:29:49,Luna Correia,carolina_aparecida@gmail.com,N/A,N/A,2021-11-10 07:29:49,31000,Carolina Aparecida,N/A,N/A,:,N/A,N/A,(11) 97048-2708,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,-,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,31001,N/A,(11) 93723-4396,Analista Administrativo,2021-11-10 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,2021-11-11 11:10:31,31001,Eduardo Rios,N/A,N/A,Outros: Contato do RH,N/A,1994-12-28 00:00:00,(11) 93723-4396,Feminino,Solteiro,Não,são paulo,N/A,N/A,N/A,Analista Administrativo,Administrativa,N/A,N/A,N/A,19000.0,N/A,Ensino Superior Incompleto,Nenhum,Nenhum,-,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,31002,N/A,(11) 92399-9824,Administrativo | Financeiro,2021-11-10 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,2021-11-10 11:42:36,31002,Pedro Henrique Carvalho,N/A,N/A,Anúncio:,N/A,1988-12-12 00:00:00,(11) 92399-9824,Feminino,Solteiro,Não,são paulo,N/A,N/A,N/A,Administrativo | Financeiro,Administrativa,N/A,"MS [77-418] MOS: Microsoft Office Word 2013, M...",N/A,25000.0,N/A,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,N/A,Administração de Empresas,2012,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,31003,N/A,(11) 98100-1727,Área administrativa,2021-11-10 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,2021-11-10 16:04:51,31003,Thiago Barbosa,N/A,N/A,Site de Empregos: Infojobs,N/A,1992-05-08 00:00:00,(11) 98100-1727,Feminino,Casado,Não,são paulo,N/A,N/A,N/A,Área administrativa,Administrativa,N/A,N/A,N/A,11000.0,N/A,Ensino Superior Incompleto,Nenhum,Nenhum,-,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,31004,N/A,(11) 92517-2678,N/A,2021-11-10 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,N/A,N/A,2021-11-10 09:22:03,31004,Diogo das Neves,N/A,N/A,:,N/A,1969-12-31 00:00:00,(11) 92517-2678,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,-,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [101]:
# Salvar o DataFrame vagas em um arquivo CSV
applicants.to_csv("applicants_processadas.csv", index=False)
print("DataFrame 'applicants' salvo como 'applicants_processadas.csv'")

DataFrame 'applicants' salvo como 'applicants_processadas.csv'


# TRATAMENTO VAGAS



In [53]:
df_raw = pd.read_json("/content/vagas.json")

In [54]:
# 1) Garantir que cada célula seja um dict (pode vir como string)
def to_dict(x):
    if isinstance(x, dict):
        return x
    if pd.isna(x):
        return {}
    s = str(x)
    # tenta Python-literal
    try:
        return ast.literal_eval(s)
    except Exception:
        pass
    # tenta JSON
    try:
        return json.loads(s)
    except Exception:
        return {}

df_raw = df_raw.applymap(to_dict)

In [55]:
# 2) Transpor: cada linha vira um job_id
# Transpose df_raw so job_id is in the index and the nested sections are columns
df_t = df_raw.T
df_t.index.name = "job_id"

In [56]:
df_t.head()

,informacoes_basicas,perfil_vaga,beneficios
job_id,,,
5185,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5184,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5183,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."
5182,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '- p/ mês (168h)', 'valor_comp..."
5181,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '..."


In [27]:
# 3) Combinar os 3 dicionários por linha com prefixos para evitar colisão de nomes
def merge_sections(row):
    base = {f"inf_{k}": v for k, v in row.get("informacoes_basicas", {}).items()}
    perf = {f"perfil_{k}": v for k, v in row.get("perfil_vaga", {}).items()}
    benef = {f"benef_{k}": v for k, v in row.get("beneficios", {}).items()}
    return {**base, **perf, **benef}


In [28]:
# 4) Flatten: expand nested dicts to columns
records = df_t.apply(merge_sections, axis=1)
df_flat = pd.DataFrame(list(records), index=df_t.index).reset_index()

In [29]:
df_flat.columns = (
    pd.Series(df_flat.columns)
    .astype(str)
    .str.strip()
    .str.lower()
    .str.normalize('NFKD')  # remove acentos
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
    .str.replace(r'\s+', '_', regex=True)
)

In [30]:
# 5) Tipagem: datas (dia primeiro), valores monetários, inteiros
# Datas: qualquer coluna que contenha 'data'
for c in [c for c in df_flat.columns if 'data' in c]:
    df_flat[c] = pd.to_datetime(df_flat[c], dayfirst=True, errors='coerce')

# Função para converter dinheiro/bruto em float
money_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['valor', 'sal', 'remun', 'preco', 'custo'])]
def money_to_float(v):
    if pd.isna(v):
        return np.nan
    s = str(v).strip()
    if s in {'-', ''}:
        return np.nan
    # remove R$, espaços e separadores de milhar; troca vírgula por ponto
    s = s.replace('R$', '').replace('r$', '').replace(' ', '').replace('.', '').replace(',', '.')
    # pega o primeiro número válido
    m = re.search(r'-?\d+(\.\d+)?', s)
    return float(m.group(0)) if m else np.nan

for c in money_like_cols:
    df_flat[c] = df_flat[c].map(money_to_float)

In [31]:
# Inteiros evidentes (ex: limites, cargas, etc.)
int_like_cols = [c for c in df_flat.columns if any(tok in c for tok in ['limite', 'carga', 'qtd', 'quant', 'horas'])]
for c in int_like_cols:
    # só converte se não for já numérico
    if not pd.api.types.is_integer_dtype(df_flat[c]):
        df_flat[c] = pd.to_numeric(df_flat[c], errors='coerce').astype('Int64')

In [32]:
# 6) String hygiene
str_cols = df_flat.select_dtypes(include=['object']).columns
df_flat[str_cols] = df_flat[str_cols].apply(lambda s: s.astype(str).str.strip().replace({'nan': np.nan}))

In [33]:
# 7) Remover colunas totalmente nulas e duplicadas
df_flat = df_flat.loc[:, df_flat.notna().any(axis=0)]
df_flat = df_flat.loc[:, ~df_flat.columns.duplicated()]

In [85]:
# 8) Ordenar colunas (id primeiro)
cols = ['job_id'] + [c for c in df_flat.columns if c != 'job_id']
df_final = df_flat[cols]

In [35]:
df_final.head()

,job_id,inf_data_requicisao,inf_titulo_vaga,inf_vaga_sap,inf_cliente,inf_solicitante_cliente,inf_empresa_divisao,inf_requisitante,inf_analista_responsavel,inf_tipo_contratacao,inf_prazo_contratacao,inf_objetivo_vaga,inf_prioridade_vaga,inf_origem_vaga,inf_superior_imediato,inf_nome,inf_telefone,perfil_pais,perfil_estado,perfil_cidade,perfil_bairro,perfil_regiao,perfil_local_trabalho,perfil_vaga_especifica_para_pcd,perfil_faixa_etaria,perfil_horario_trabalho,perfil_nivel_profissional,perfil_nivel_academico,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_outro_idioma,perfil_areas_atuacao,perfil_principais_atividades,perfil_competencia_tecnicas_e_comportamentais,perfil_demais_observacoes,perfil_viagens_requeridas,perfil_equipamentos_necessarios,benef_valor_venda,benef_valor_compra_1,benef_valor_compra_2,inf_data_inicial,inf_data_final,perfil_habilidades_comportamentais_necessarias,inf_nome_substituto
0,5185,2021-05-04,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,NaN,NaN,NaT,NaT,NaN,NaN
1,5184,2021-05-04,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,5183,2021-05-04,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,NaN,NaN,NaT,NaT,NaN,NaN
3,5182,2021-05-04,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,168.0,NaN,NaN,2021-05-18,2022-01-17,NaN,NaN
4,5181,2021-05-04,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [36]:
df_final.isnull().sum()

,0
job_id,0
inf_data_requicisao,0
inf_titulo_vaga,0
inf_vaga_sap,0
inf_cliente,0
inf_solicitante_cliente,0
inf_empresa_divisao,0
inf_requisitante,0
inf_analista_responsavel,0
inf_tipo_contratacao,0


In [37]:
pd.set_option('display.max_columns', None)
df_final.tail(5)

,job_id,inf_data_requicisao,inf_titulo_vaga,inf_vaga_sap,inf_cliente,inf_solicitante_cliente,inf_empresa_divisao,inf_requisitante,inf_analista_responsavel,inf_tipo_contratacao,inf_prazo_contratacao,inf_objetivo_vaga,inf_prioridade_vaga,inf_origem_vaga,inf_superior_imediato,inf_nome,inf_telefone,perfil_pais,perfil_estado,perfil_cidade,perfil_bairro,perfil_regiao,perfil_local_trabalho,perfil_vaga_especifica_para_pcd,perfil_faixa_etaria,perfil_horario_trabalho,perfil_nivel_profissional,perfil_nivel_academico,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_outro_idioma,perfil_areas_atuacao,perfil_principais_atividades,perfil_competencia_tecnicas_e_comportamentais,perfil_demais_observacoes,perfil_viagens_requeridas,perfil_equipamentos_necessarios,benef_valor_venda,benef_valor_compra_1,benef_valor_compra_2,inf_data_inicial,inf_data_final,perfil_habilidades_comportamentais_necessarias,inf_nome_substituto
14076,12368,2024-02-28,PMP 5203658 - Dynamics,Não,Miller-Curry,Guilherme Campos,Decision Campinas,Sra. Juliana Oliveira,Sra. Joana Sousa,CLT Full,Indeterminado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,,De: Até:,,Analista,Ensino Superior Completo,Nenhum,,,TI - Desenvolvimento/Programação-,Dynamics,Dynamics,,,,123.0,9.0,NaN,NaT,NaT,NaN,NaN
14077,12367,2024-02-28,MM com IM - FSR 3910,Não,Bishop-Reed,Lívia Vasconcelos,Decision São Paulo,Manuella Jesus,Srta. Bella Ferreira,"CLT Full, PJ/Autônomo",Indeterminado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,,De: Até:,,Analista,Ensino Superior Completo,Avançado,Básico,,TI - Projetos-,1 Deve ter boa experiência em configuração de ...,inglês avançado: (tem que se comunicar com o G...,FSR: 3910,Não,,168.0,NaN,NaN,NaT,NaT,NaN,NaN
14078,12366,2024-02-27,964 - Assistente fiscal ou financeiro,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Maria Clara Pires,"CLT Full, PJ/Autônomo",Indeterminado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,Português Básico,TI - Projetos-,Qualificações técnicas: conhecimento básico em...,"Local: Alphaville, Barueri\nQual região: SP\nI...","Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168.0,NaN,NaN,NaT,NaT,NaN,NaN
14079,12365,2024-02-27,966 - Analista pleno ou sênior de área fiscal,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Melina Montenegro,"CLT Full, PJ/Autônomo",Indeterminado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,Barueri,,Grande São Paulo,2000,,De: Até:,,Sênior,Ensino Superior Completo,Básico,Básico,Português Básico,TI - Projetos-,Qualificações técnicas: apuração de impostos r...,"Local: Alphaville, Barueri, SP.\nQual região: ...","Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168.0,NaN,NaN,NaT,NaT,NaN,NaN
14080,12364,2024-02-27,965 - Assistente ou analista de área contábil,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Laura Pacheco,"CLT Full, PJ/Autônomo",Indeterminado,Contratação,Alta: Alta complexidade 3 a 5 dias,Nova Posição,Superior Imediato:,,,Brasil,São Paulo,Barueri,,Grande São Paulo,2000,,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,Português Básico,TI - Projetos-,Qualificações técnicas: reconciliação de conta...,"Local: Alphaville, Barueri, SP.\nRegião: Grand...","Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168.0,NaN,NaN,NaT,NaT,NaN,NaN


In [38]:
vagas = df_final[["job_id", "inf_data_requicisao", "inf_titulo_vaga", "inf_vaga_sap", "inf_cliente", "inf_solicitante_cliente", "inf_empresa_divisao", "inf_requisitante","inf_analista_responsavel", "inf_tipo_contratacao",
                 "inf_prazo_contratacao", "inf_prioridade_vaga", "inf_superior_imediato", "perfil_estado", "perfil_cidade", "perfil_vaga_especifica_para_pcd", "perfil_nivel_profissional", "perfil_nivel_academico",
                 "perfil_nivel_ingles", "perfil_nivel_espanhol", "perfil_areas_atuacao", "perfil_principais_atividades", "perfil_competencia_tecnicas_e_comportamentais", "perfil_demais_observacoes", "perfil_viagens_requeridas",
                 "perfil_equipamentos_necessarios", "benef_valor_venda"]]

In [39]:
vagas.head()

,job_id,inf_data_requicisao,inf_titulo_vaga,inf_vaga_sap,inf_cliente,inf_solicitante_cliente,inf_empresa_divisao,inf_requisitante,inf_analista_responsavel,inf_tipo_contratacao,inf_prazo_contratacao,inf_prioridade_vaga,inf_superior_imediato,perfil_estado,perfil_cidade,perfil_vaga_especifica_para_pcd,perfil_nivel_profissional,perfil_nivel_academico,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_areas_atuacao,perfil_principais_atividades,perfil_competencia_tecnicas_e_comportamentais,perfil_demais_observacoes,perfil_viagens_requeridas,perfil_equipamentos_necessarios,benef_valor_venda
0,5185,2021-05-04,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,Superior Imediato:,São Paulo,São Paulo,Não,Sênior,Ensino Superior Completo,Avançado,Fluente,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN
1,5184,2021-05-04,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,,Superior Imediato:,São Paulo,São Paulo,Não,Sênior,Ensino Superior Completo,Fluente,Nenhum,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN
2,5183,2021-05-04,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,,Superior Imediato:,São Paulo,São Paulo,Não,Analista,Ensino Superior Completo,Nenhum,Intermediário,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN
3,5182,2021-05-04,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Alta: Alta complexidade 3 a 5 dias,Superior Imediato:,São Paulo,São Paulo,Não,Analista,Ensino Superior Completo,Básico,Básico,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,168.0
4,5181,2021-05-04,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,Superior Imediato:,São Paulo,São Paulo,Não,Sênior,Ensino Superior Completo,Intermediário,Nenhum,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN


In [41]:
vagas["inf_superior_imediato"] = vagas["inf_superior_imediato"].fillna('N/A')
vagas["perfil_equipamentos_necessarios"] = vagas["perfil_equipamentos_necessarios"].fillna('Nao Necessario')
vagas["benef_valor_venda"] = vagas["benef_valor_venda"].fillna(0)
vagas["inf_prazo_contratacao"] = vagas["inf_prazo_contratacao"].fillna('N/A')
vagas["inf_prioridade_vaga"] = vagas["inf_prioridade_vaga"].fillna('N/A')

vagas["inf_superior_imediato"] = vagas["inf_superior_imediato"].replace({'': 'N/A', 'nan': 'N/A'})
vagas["perfil_equipamentos_necessarios"] = vagas["perfil_equipamentos_necessarios"].replace({'-': 'Nao Necessario', 'nan': 'Nao Necessario'})
vagas["inf_prazo_contratacao"] = vagas["inf_prazo_contratacao"].replace({'': 'N/A', 'nan': 'N/A'})
vagas["inf_prioridade_vaga"] = vagas["inf_prioridade_vaga"].replace({'': 'N/A', 'nan': 'N/A'})

In [44]:
# Salvar o DataFrame vagas em um arquivo CSV
vagas.to_csv("vagas_processadas.csv", index=False)
print("DataFrame 'vagas' salvo como 'vagas_processadas.csv'")

DataFrame 'vagas' salvo como 'vagas_processadas.csv'
